In [3]:
# Importing general python modules
import numpy as np
import astropy.constants as const
import astropy.units as u
# Plotting packages
import matplotlib.pyplot as plt
import matplotlib
# Importing the modules to be used in the class.
from GalaxyMass import ComponentMass
from CenterOfMass import CenterOfMass
from IPython.display import Latex


In [10]:
class M33AnalyticOrbit:
    
    def __init__(self,filename):
        ''' Initiating the class.
        INPUTS: output filename (defined below)'''
        
        # Instantiating G's value without units.
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        # defining the output file.
        self.filename = '%s.txt'%(filename)
        
        # Instantiating CenterOfMass for M33.
        M33 = CenterOfMass('Orbit_M33.txt',2)
        
        # Calling the COM_P and COM_V functions to find the center of mass pos/vel of M33
        self.M33r = M33.COM_P(0.1)
        self.M33v = M33.COM_V(self.M33r[0],self.M33r[1],self.M33r[2])
        
        # Instantiating center of mass for M31.
        M31 = CenterOfMass('Orbit_M31.txt',2)
        
        # Calling COM_P and COM_V functions in CenterOfMass to calculate pos/vel of M31.
        self.M31r = M31.COM_P(0.1)
        self.M31v = M31.COM_V(self.M31r[0],self.M31r[1],self.M31r[2])
        
        # Difference of the po/vel vectors of M33 and M31.
        self.r0 = np.array(self.M33r) - np.array(self.M31r)
        self.v0 = np.array(self.M33v) - np.array(self.M31v)
        
        # scale length for disk
        self.rdisk = 5
        # calling the component mass function to calculate the disk mass (ptype = 2)
        self.Mdisk = ComponentMass('M31_000.txt',2)
        # scale length of bulge.
        self.rbulge = 1
        # calculating the bulge mass (ptype = 3)
        self.Mbulge = ComponentMass('M31_000.txt',3)
        # scale radius for halo
        self.rhalo = 62
        # calculating the halo mass (ptype = 1)
        self.Mhalo = ComponentMass('M31_000.txt',1)
        
        
        
        def HernquistAccel(self, M, r_a, x, y, z):
            '''Function to calculate the gravitational acceleration of bulge and halo mass
            through the Hernquist profile.
            INPUTS: mass (bulge or halo) in M_sun, scale radius (r_a), relative distance between the galaxies
            represented by the vector (x,y,z)
            RETURNS: The gravitational acceleration VECTOR.'''
            # defining the x,y,z coordinates of the M33-M31 distance.
            x = np.absolute(self.r0[0])
            y = np.absolute(self.r0[1])
            z = np.absolute(self.r0[2])
            
            # calculating the gravitational acceleration.
            a_x = (self.G)*M/r/(r+r_a)**2*x
            a_y = (self.G)*M/r/(r+r_a)**2*y
            a_z = (self.G)*M/r/(r+r_a)**2*z
            # assigning the components of the acceleration to an array (aBulgeHalo)
            aBulgeHalo = np.array([a_x,a_y,a_z])
            return aBulgeHalo
        
        def MiyamotoNagaiAccel(self, M, rd, x, y, z):
            ''' Function to calculate the gravitational acceleration of disk particles through the Miyamoto
            Nagai profile. 
            INPUTS: mass (in M_sun), scaling length (r_disk), relative distance between the galaxies, 
            represented by a vector.
            RETURNS: the gravitational acceleration of disk particles.'''
            # Same as in HernquistAccel
            x = np.absolute(self.r0[0])
            y = np.absolute(self.r0[1])
            z = np.absolute(self.r0[2])
            # defining extra terms required for the calculation.
            rd = self.rdisk
            zd = rd.value/5.0
            R = np.sqrt(x**2 + y**2)
            B = rd.value + np.sqrt(z**2 + zd**2)
            
            # calculating the x,y and z components of the acceleration.
            a_x = -self.G*M/(R**2 + B**2)**1.5*x
            a_y = -self.G*M/(R**2 + B**2)**1.5*y
            
            az1 = -self.G*M/(R**2 + B**2)**1.5*z
            a_z = az1*B/np.sqrt(z**2 + zd**2)
            
            # assigning the components of the acceleration to an a array (aDisk)
            aDisk = np.array([a_x,a_y,a_z])
            return aDisk
        
        def M31Accel(self, x, y, z):
            ''' Function to calculate the total acceleration VECTOR of M31.
            INPUTS: (x,y,z) coordinates of the M33-M31 distance.
            RETURNS: the total acceleration of M31.'''
            
            aTotal = self.Hernquist(M,r_a,x,y,z) + self.MiyamotoNagaiAccel(M,rd,x,y,z)
            return aTotal
        
        def LeapFrog(self,dt,x,y,z,vx,vy,vz):
            '''defining a function that calculates the relative positions and velocities of the galaxies
            at various timesteps.
            INPUTS: time interval (dt), distance between the two galaxies (x,y,z) and relative velocities
            of the two galaxies (vx,vy,vz)
            RETURNS: the relative position and velocity at full timestep. '''
            
            # Refer to the HW-7 guidelines where these formulae are laid out.
            x_half = x + self.vx*dt/2
            y_half = y + self.vy*dt/2
            z_half = z + self.vz*dt/2
            # assigning the components to an array.
            r_half = np.array([x_half,y_half,z_half])
            
            v_full = v + self.M31Accel(x_half,y_half,z_half)*dt
            
            r_full = r_half + v_full*dt/2
            
            return r_full,v_full
            
        def OrbitIntegrator(self,t0,dt,tmax):
            '''defining a function that loops over the LeapFrog function and updates the kinematics of 
            galaxies at various times.
            INPUTS: starting time (t0), ending time (tmax), time interval (dt)
            RETURNS: a file containing the relative distances and velocities of M33-M31 system at various 
            times.'''
            
            t = t0
            # initializing an array
            Orbit = np.zeros([int(tmax/dt)+2, 7])
            # defining the first row of the array.
            Orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
            # initializing array elements
            i = 1
                
            while t<=tmax: # as long as t does not cross tmax:
                t += dt # update time by dt  
                Orbit[i,1] = t # the first column of the array will have time
                r,v = self.LeapFrog(dt,x,y,z,vx,vy,vz) # update the positions and velocites.
                # store the positions ans velocities in the respective columns.
                Orbit[i,1:4] = r
                Orbit[i,4:7] = v
                # update the array element.
                i += 1
                
            # save the output file.        
            np.savetxt(self.filename,Orbit,fmt = "%11.3f"*7, comments='#',
                        header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                              .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))


                
            
            
            
            
        
        

In [11]:
M33AnalyticOrbit('Integrated_Orbit')

ValueError: too many values to unpack (expected 2)